In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from operator import itemgetter
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import CharacterTextSplitter

In [9]:
load_dotenv()

True

In [10]:
llm=ChatOpenAI(model="gpt-4o-mini",temperature=0,openai_api_key=os.getenv("OPENAI_API_KEY"))

In [11]:
loader=PyMuPDFLoader("info.pdf")
docs=loader.load()

text_splitter=CharacterTextSplitter(
    separator="\n\n",
    chunk_size=500,
    chunk_overlap=50,
    length_function=len
)

chunks=[]
for page in docs:
    split_text=text_splitter.split_text(page.page_content)
    chunks.extend(split_text)

In [13]:
template = '''
    Please provide a concise summary of the text below:
    {text}
'''

prompt=ChatPromptTemplate.from_template(template)

In [19]:
query_fetcher= itemgetter("text")

setup={"text":query_fetcher}

In [26]:
#chain = setup | prompt | llm
chain=llm
batch=[]
for i in range(len(chunks)):
    answer=prompt.invoke({"text":chunks[i]})
    batch.append(answer.messages[0].content)

chain.batch(batch[:5])

[AIMessage(content='FAST NUCES (National University of Computer and Emerging Sciences) is a premier university in Pakistan, established in July 2000, with five modern campuses in major cities. It is known for its high-quality education and significant impact on local industries, particularly in software development. The university supports over 11,000 students, including a substantial number of female students, and employs more than 500 skilled faculty members. FAST NUCES emphasizes professional excellence and responsible citizenship, providing a peaceful learning environment with extensive recreational facilities. It offers financial aid, including scholarships and interest-free loans, benefiting over 2,000 students. The university aims to enhance its global research recognition in the coming decade and offers a Doctor of Philosophy in Management Sciences among other programs.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_t

In [48]:
batches=[]

for i in range(len(chunks)):
    batches.append({"text":chunks[i]})

In [49]:
response=chain.batch(batches[:5])

In [50]:
response

[AIMessage(content='FAST NUCES (National University of Computer and Emerging Sciences) is a premier university in Pakistan, established in July 2000, with five modern campuses in major cities. It is known for its high-quality education and significant impact on local industries, particularly in software development. The university supports over 11,000 students, including a substantial number of female students, with more than 500 skilled faculty members. FAST NUCES emphasizes professional excellence and responsible citizenship, providing a peaceful learning environment and recreational facilities. It offers financial aid, including scholarships and interest-free loans, benefiting over 2,000 students. The university aims to enhance its global research recognition in the coming decade and offers a Doctor of Philosophy in Management Sciences among other programs.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 148, 'prompt_tokens': 385, 'tota